In [24]:
""" Water Stress in delta basins vs individual basins.
-------------------------------------------------------------------------------

Steps:
1. Read data from RDS
1. Convert to WKT
1. Create BQ Dataset
1. Upload to BQ (geom in string type)
1. Create new table with geometry type in BQ


"""

TESTING = 0
OVERWRITE_OUTPUT = 1
SCRIPT_NAME = 'Y2018M08D02_RH_Hybas_RDS_To_BQ_V01'
OUTPUT_VERSION = 1

BQ_PROJECT_ID = "aqueduct30"
BQ_INPUT_DATASET_NAME = "aqueduct30v01"
BQ_OUTPUT_DATASET_NAME = "spatial_test"
BQ_OUTPUT_TABLE_NAME = "{}_v{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION).lower()

RDS_DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
RDS_DATABASE_NAME = "database01"
RDS_INPUT_TABLE_NAME = "hybas06_v04"


print("\nRDS_DATABASE_ENDPOINT: ", RDS_DATABASE_ENDPOINT,
      "\nRDS_DATABASE_NAME: ", RDS_DATABASE_NAME,
      "\nRDS_INPUT_TABLE_NAME: ",RDS_INPUT_TABLE_NAME,
      "\nBQ_OUTPUT_DATASET_NAME: ", BQ_OUTPUT_DATASET_NAME,
      "\nBQ_OUTPUT_TABLE_NAME: ", BQ_OUTPUT_TABLE_NAME)



RDS_DATABASE_ENDPOINT:  aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com 
RDS_DATABASE_NAME:  database01 
RDS_INPUT_TABLE_NAME:  hybas06_v04 
BQ_OUTPUT_DATASET_NAME:  spatial_test 
BQ_OUTPUT_TABLE_NAME:  y2018m08d02_rh_hybas_rds_to_bq_v01_v01


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M08D02 UTC 13:57


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [9]:
import os
import sqlalchemy
import pandas as pd
import geopandas as gpd
from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/.google.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "aqueduct30"
client = bigquery.Client(project=BQ_PROJECT_ID)

In [4]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

engine = sqlalchemy.create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,RDS_DATABASE_ENDPOINT,RDS_DATABASE_NAME))


In [5]:
sql = """
SELECT
  pfaf_id,
  geom,
  ST_AsText(geom) AS wkt
FROM
  {}
""".format(RDS_INPUT_TABLE_NAME)


In [6]:
gdf = gpd.read_postgis(sql=sql,
                       con=engine)

In [7]:
gdf.head()

,pfaf_id,geom,wkt
0,111015,(POLYGON ((34.00000000000003 26.62500000000002...,"MULTIPOLYGON(((34 26.625,34.00242445204 26.624..."
1,111019,(POLYGON ((36.24444444444446 23.55833333333337...,MULTIPOLYGON(((36.2444444444445 23.55833333333...
2,111020,(POLYGON ((35.14166666666669 22.58333333333336...,MULTIPOLYGON(((35.1416666666667 22.58333333333...
3,111041,(POLYGON ((36.52916666666668 21.25000000000003...,"MULTIPOLYGON(((36.5291666666667 21.25,36.54409..."
4,111050,(POLYGON ((37.19583333333335 20.00416666666668...,MULTIPOLYGON(((37.1958333333333 20.00416666666...


In [14]:
df = pd.DataFrame(gdf.drop("geom",1))

,pfaf_id,wkt
0,111015,"MULTIPOLYGON(((34 26.625,34.00242445204 26.624..."
1,111019,MULTIPOLYGON(((36.2444444444445 23.55833333333...
2,111020,MULTIPOLYGON(((35.1416666666667 22.58333333333...
3,111041,"MULTIPOLYGON(((36.5291666666667 21.25,36.54409..."
4,111050,MULTIPOLYGON(((37.1958333333333 20.00416666666...


In [25]:
# Storing in separate dataset

dataset_ref = client.dataset(BQ_OUTPUT_DATASET_NAME)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = 'US'
dataset = client.create_dataset(dataset)

In [26]:
destination_table = "{}.{}".format(BQ_OUTPUT_DATASET_NAME,BQ_OUTPUT_TABLE_NAME)

In [27]:
print(destination_table)

spatial_test.y2018m08d02_rh_hybas_rds_to_bq_v01_v01


In [21]:
df = df[1:10]

In [28]:
df.to_gbq(destination_table=destination_table,
          project_id=BQ_PROJECT_ID,
          chunksize=100,
          if_exists="replace")


0it [00:00, ?it/s]
1it [00:03,  3.55s/it]

In [29]:
# woohoo it works but visualization sucks in BQ. back to square 1